In [2]:
import pandas as pd
import mysql.connector
import hashlib
from datetime import datetime

In [3]:
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="0007",
        database="client_query_db"
    )

In [1]:
def insert_queries_from_csv(csv_path="C:/Users/saran/Downloads/sample_queries_250.csv"):
    if "csv_inserted" in st.session_state and st.session_state["csv_inserted"]:
        return  # already inserted

    conn = get_connection()
    cursor = conn.cursor()
    now = datetime.now()
    df = pd.read_csv(csv_path)
    for _, row in df.iterrows():
        email = row.get("email")
        mobile = str(row.get("mobile")) if pd.notna(row.get("mobile")) else None
        heading = row.get("query_heading")
        description = row.get("query_description")
        status = row.get("status", "Open")
        query_created_time = pd.to_datetime(row.get("query_created_time"))
        query_closed_time = pd.to_datetime(row.get("query_closed_time")) if pd.notna(row.get("query_closed_time")) else None
        cursor.execute("""SELECT id FROM queries 
            WHERE email=%s AND query_heading=%s AND query_description=%s AND query_created_time=%s
        """, (email, heading, description, query_created_time)) # Skip duplicates (ignore query_closed_time in WHERE)
        if cursor.fetchone():
            continue
        cursor.execute("""INSERT INTO queries 
            (email, mobile, query_heading, query_description, query_created_time, status, query_closed_time)
            VALUES (%s, %s, %s, %s, %s, %s, %s)""", (email,mobile,heading,description,query_created_time,status,query_closed_time)) # Inserting query  
        conn.commit()
    cursor.close()
    conn.close()
    print("csv_inserted")